# 1. 국토교통부 실거래가 공개시스템 데이터 분석
- 대상 데이터 - 아파트(매매) 실거래가 2025년 12월 1달 기준
- 본 분석에서는 ㎡당 가격 기준으로 지역별 주택 가격 수준을 비교하기 위해 아파트 매매 실거래가 데이터를 우선적으로 활용
- 고려사항
    - **실제 청년층의 주거 이동은 아파트 매매에 국한되지 않으며, 전·월세 거래, 연립·다세대, 오피스텔, 단독·다가구 주택 등 다양한 주택 유형과 거래 형태가 혼재되어 있어, 이를 고려하여 이후 분석에서는 거래 유형 및 주택 유형을 확장할 필요가 있음.**

In [ ]:
import pandas as pd

df = pd.read_csv(
    "./data/아파트(매매)_실거래가_202512_20260128.csv",
    encoding="cp949",
    sep=",",
    skiprows=15,
    engine="python",
)

df.head()

In [7]:
df.info()

In [8]:
df.describe()

In [81]:
# 거래금액 전처리 str -> int
df["거래금액(만원)"] = (
    df["거래금액(만원)"].str.replace(",", "", regex=False).astype(int)
)

In [82]:
# 컬럼 신규 생성
df["㎡당가격"] = df["거래금액(만원)"] / df["전용면적(㎡)"]

In [83]:
# “청년 이동” 분석인데 84㎡, 120㎡ 섞이면 논리 무너짐
df_young = df[(df["전용면적(㎡)"] >= 40) & (df["전용면적(㎡)"] <= 60)].copy()

In [84]:
# 시군구까지만
df_young["시군구_정제"] = df_young["시군구"].str.split().str[:3].str.join(" ")

In [85]:
# 지역별 주거비 대표값 생성
region_price = (
    df_young.groupby("시군구_정제")
    .agg(
        평균거래금액=("거래금액(만원)", "mean"),
        평균_면적당가격=("㎡당가격", "mean"),
        거래건수=("NO", "count"),
    )
    .reset_index()
)

region_price = region_price.rename(
    columns={
        "avg_price": "평균거래금액",
        "avg_price_per_m2": "평균_㎡당가격",
        "cnt": "거래건수",
    }
)

In [9]:
df.head()

In [ ]:
# 지역별 평균 ㎡당 가격 분포
region_price["평균_면적당가격"].describe()

In [87]:
# 거래건수 필터링
region_price_f = region_price[region_price["거래건수"] >= 30]

In [ ]:
# ㎡당 가격 기준 상위 10
cheap_top10 = region_price_f.sort_values("평균_면적당가격").head(10)
cheap_top10

In [ ]:
# ㎡당 가격 기준 하위 10
expensive_top10 = region_price_f.sort_values("평균_면적당가격", ascending=False).head(
    10
)
expensive_top10

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 정렬 (낮은 값 → 위, 높은 값 → 아래)
plot_df = expensive_top10.sort_values("평균_면적당가격")

plt.figure(figsize=(8, 6))
plt.barh(plot_df["시군구_정제"], plot_df["평균_면적당가격"])

plt.title("㎡당 가격 상위 10 지역", fontsize=13, fontweight="bold")
plt.xlabel("㎡당 평균 가격")
plt.ylabel("")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 정렬 (낮은 값 → 위, 높은 값 → 아래)
plot_df = cheap_top10.sort_values("평균_면적당가격")

plt.figure(figsize=(8, 6))
plt.barh(plot_df["시군구_정제"], plot_df["평균_면적당가격"])

plt.title("㎡당 가격 하위 10 지역", fontsize=13, fontweight="bold")
plt.xlabel("㎡당 평균 가격")
plt.ylabel("")

plt.tight_layout()
plt.show()

# 2. 청년 순이동률
- 비수도권 시·군을 대상으로 청년 순이동의 ‘절대 규모(체급)’와 ‘비율(체질)’을 결합한 복합 흡수력 지수를 설계하고 랭킹화한 분석

- 청년이 계속 모이는 구조를 가진 지역 TOP5
    1. 아산시
    2. 완주군
    3. 청주시
    4. 춘천시
    5. 당진시

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [180]:
import pandas as pd
import requests

admin = pd.read_csv("../../data/행정구역코드.csv")
admin["행정구역코드"] = admin["행정구역코드"].astype(str)

# 수도권 + 광역시 제외
exclude_prefix = ["11", "31", "21", "22", "23", "24", "25", "26"]
sigun = admin[~admin["행정구역코드"].str[:2].isin(exclude_prefix)].copy()

objL1 = "+".join(sigun["행정구역코드"].tolist()) + "+"


In [181]:
url = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": "ZWRiNzEyMzEwNGI5OWQ3NzcxNGM1MDNiOGJkOTQ0Y2M=",
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "itmId": "T001+",
    "objL1": objL1,
    "objL2": "ALL",
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "Y",
    "newEstPrdCnt": "3",  # 최근 3개년
}

response = requests.get(url, params=params)
df = pd.DataFrame(response.json())

In [ ]:
df.head()

In [183]:
# 지표 분리 (율 vs 명)
rate_df = df[df["C2_NM"].str.contains("순이동률")].copy()
count_df = df[
    df["C2_NM"].str.contains("순이동") & ~df["C2_NM"].str.contains("률")
].copy()

In [184]:
# 컬럼명 의미 고정
rate_df.rename(columns={"DT": "청년순이동률"}, inplace=True)
count_df.rename(columns={"DT": "청년순이동"}, inplace=True)

In [185]:
# 타입 정리
for d in (rate_df, count_df):
    d["PRD_DE"] = d["PRD_DE"].astype(int)

rate_df["청년순이동률"] = pd.to_numeric(rate_df["청년순이동률"], errors="coerce")
count_df["청년순이동"] = pd.to_numeric(count_df["청년순이동"], errors="coerce")

In [186]:
# 같은 지역 · 같은 연도 기준 병합
merged = rate_df.merge(
    count_df[["PRD_DE", "C1", "청년순이동"]], on=["PRD_DE", "C1"], how="inner"
)

In [187]:
# 연도 정렬 (이거 안 하면 iloc 논리 깨짐)
merged = merged.sort_values(["C1", "PRD_DE"])

In [ ]:
# 1·2·3년 요약
summary = (
    merged.groupby(["C1", "C1_NM"])
    .agg(
        순이동_최근1년=("청년순이동", lambda x: x.iloc[-1]),
        순이동_최근2년=("청년순이동", lambda x: x.iloc[-2:].sum()),
        순이동_최근3년=("청년순이동", "sum"),
        평균_청년순이동률=("청년순이동률", "mean"),
    )
    .reset_index()
)

summary

In [ ]:
df["C1_NM"].unique()

In [ ]:
# 체급 : 절대량, 청년이 몇 명이나 순유입/순유출 되었나 -> 어디에 사람이 많이 움직였나?
# 체질 : 비율·구조, 그 지역 인구 대비 청년을 끌어당기는 힘 -> 이 지역은 청년에게 매력적인 구조인가?
# 청년흡수지수 : 종합 평가용, 체급(60%) + 체질(40%)

import numpy as np

df = summary.copy()

# 1. 실제 컬럼명 기준으로 선택
df_use = df[
    [
        "C1",  # 행정코드
        "C1_NM",  # 지역명
        "순이동_최근3년",
        "평균_청년순이동률",
    ]
].copy()

# 2. 표준화 (z-score)
df_use["체급_z"] = (
    df_use["순이동_최근3년"] - df_use["순이동_최근3년"].mean()
) / df_use["순이동_최근3년"].std()

df_use["체질_z"] = (
    df_use["평균_청년순이동률"] - df_use["평균_청년순이동률"].mean()
) / df_use["평균_청년순이동률"].std()

# 3. 청년 흡수 지수
df_use["청년흡수지수"] = 0.6 * df_use["체급_z"] + 0.4 * df_use["체질_z"]

# 4. 랭킹
ranked = df_use.sort_values("청년흡수지수", ascending=False).reset_index(drop=True)

ranked


In [ ]:
df_use["C1_NM"].unique()

## 2-1. 문화/레저 인프라 분석

### 2-1-1. 인구 십만명당 문화기반시설수(시도/시/군/구)

In [ ]:
# 인구 십만명당 문화기반시설수(시도/시/군/구)
# https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1YL20931&conn_path=I2

import requests
import pandas as pd
import os

admin = pd.read_csv("../../data/행정구역코드.csv")
admin["행정구역코드"] = admin["행정구역코드"].astype(str)

# 수도권 + 광역시 제외
exclude_prefix = ["11", "31", "21", "22", "23", "24", "25", "26"]
sigun = admin[~admin["행정구역코드"].str[:2].isin(exclude_prefix)].copy()

objL1 = "+".join(sigun["행정구역코드"].tolist()) + "+"

url = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": os.getenv("KOSIS_API_KEY"),
    "orgId": "101",
    "tblId": "DT_1YL20931",
    "itmId": "T001+",
    "objL1": objL1,
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "Y",
    "newEstPrdCnt": "1",  # 최근 1개 연도 -> 2024년 기준
}

res = requests.get(url, params=params)
print(res.request.url)

data = res.json()
data

culture_df = pd.DataFrame(data)
culture_df.head(20)

In [ ]:
culture_df.info()

In [97]:
culture_df["DT"] = culture_df["DT"].astype(int)

In [ ]:
culture_df.info()

In [ ]:
set(ranked["C1_NM"]) & set(culture_df["C1_NM"])

In [ ]:
culture_df[["C1_NM", "DT"]]

In [ ]:
target = ["아산시", "완주군", "청주시", "춘천시", "당진시"]
filtered_df = df[df["C1_NM"].isin(target)][["C1_NM", "DT"]]
filtered_df

In [ ]:
analysis_df = pd.merge(ranked, culture_df, on="C1_NM", how="inner")
analysis_df.head()

In [ ]:
# 산점도 그래프 - 평균 청년순이동률 이용
import matplotlib.pyplot as plt

# 1. 청년 순이동률 (시 단위)
youth_df = summary[["C1_NM", "평균_청년순이동률"]].copy()

# 문화기반시설 데이터에 시 데이터만 존재해서 청년 순이동률 데이터에서 군 제거
youth_df = youth_df[youth_df["C1_NM"].str.endswith("시")]

# 2. 문화기반시설 (시 단위)
culture_use = culture_df.copy()

culture_use = culture_use[culture_use["C1_NM"].str.endswith("시")]
culture_use = culture_use.rename(columns={"DT": "문화기반시설_10만명당"})

# 3. 병합 (교집합만)
analysis_df = youth_df.merge(culture_use, on="C1_NM", how="inner")

# 4. 산점도
plt.figure(figsize=(12, 8))
plt.scatter(analysis_df["문화기반시설_10만명당"], analysis_df["평균_청년순이동률"])

for _, r in analysis_df.iterrows():
    plt.text(r["문화기반시설_10만명당"], r["평균_청년순이동률"], r["C1_NM"], fontsize=8)

plt.axhline(0, linestyle="--")
plt.axvline(analysis_df["문화기반시설_10만명당"].mean(), linestyle="--")

plt.xlabel("인구 10만명당 문화기반시설 수")
plt.ylabel("평균 청년 순이동률")
plt.title("문화기반시설과 청년 순이동률")

plt.show()

In [ ]:
# 산점도 그래프 - 청년흡수지수 이용
import matplotlib.pyplot as plt

# 1. 청년 순이동률 (시 단위)
youth_df = ranked[["C1_NM", "청년흡수지수"]].copy()

# 문화기반시설 데이터에 시 데이터만 존재해서 청년 순이동률 데이터에서 군 제거
youth_df = youth_df[youth_df["C1_NM"].str.endswith("시")]

# 2. 문화기반시설 (시 단위)
culture_use = culture_df.copy()

culture_use = culture_use[culture_use["C1_NM"].str.endswith("시")]
culture_use = culture_use.rename(columns={"DT": "문화기반시설_10만명당"})

# 3. 병합 (교집합만)
analysis_df = youth_df.merge(culture_use, on="C1_NM", how="inner")

# 4. 산점도
plt.figure(figsize=(12, 8))
plt.scatter(analysis_df["문화기반시설_10만명당"], analysis_df["청년흡수지수"])

for _, r in analysis_df.iterrows():
    plt.text(r["문화기반시설_10만명당"], r["청년흡수지수"], r["C1_NM"], fontsize=8)

plt.axhline(0, linestyle="--")
plt.axvline(analysis_df["문화기반시설_10만명당"].mean(), linestyle="--")

plt.xlabel("인구 10만명당 문화기반시설 수")
plt.ylabel("청년흡수지수")
plt.title("문화기반시설과 청년흡수지수")
plt.show()

### 2-1-2. 2025 전국 문화기반시설 총람 - 공공도서관

In [ ]:
# 공공도서관
import pandas as pd

library_df = pd.read_excel(
    "./data/2025 전국 문화기반시설 총람.xlsx",
    sheet_name="공공도서관",
    skiprows=2,
    header=3,
)

library_df.head()

In [ ]:
library_df = (
    library_df.groupby(["Unnamed: 1", "Unnamed: 2"]).size().reset_index(name="개수")
)
library_df

In [ ]:
# 산점도 그래프
import matplotlib.pyplot as plt

# 1. 청년 순이동률
youth_df = ranked[["C1_NM", "청년흡수지수"]].copy()

# 2. 문화기반시설
library_df = library_df.rename(columns={"Unnamed: 2": "C1_NM"})
library_df = library_df.rename(columns={"개수": "공공도서관 개수"})

# 3. 병합 (교집합만)
analysis_df = youth_df.merge(library_df, on="C1_NM", how="inner")

# 4. 산점도
plt.figure(figsize=(12, 8))
plt.scatter(analysis_df["공공도서관 개수"], analysis_df["청년흡수지수"])

for _, r in analysis_df.iterrows():
    plt.text(r["공공도서관 개수"], r["청년흡수지수"], r["C1_NM"], fontsize=8)

plt.axhline(0, linestyle="--")
plt.axvline(analysis_df["공공도서관 개수"].mean(), linestyle="--")

plt.xlabel("공공도서관 수")
plt.ylabel("청년흡수지수")
plt.title("공공도서관 수와 청년흡수지수")

plt.show()

### 2-1-3. 2025 전국 문화기반시설 총람 - 생활문화센터

In [ ]:
# 생활문화센터
import pandas as pd

culture_center_df = pd.read_excel(
    "./data/2025 전국 문화기반시설 총람.xlsx",
    sheet_name="생활문화센터",
    skiprows=2,
    header=2,
)

culture_center_df.head()

In [ ]:
culture_center_df = (
    culture_center_df.groupby(["Unnamed: 1", "Unnamed: 2"])
    .size()
    .reset_index(name="개수")
)
culture_center_df

In [ ]:
# 산점도 그래프
import matplotlib.pyplot as plt

# 1. 청년 순이동률
youth_df = ranked[["C1_NM", "청년흡수지수"]].copy()

# 2. 문화기반시설
culture_center_df = culture_center_df.rename(columns={"Unnamed: 2": "C1_NM"})
culture_center_df = culture_center_df.rename(columns={"개수": "생활문화센터 개수"})

# 3. 병합 (교집합만)
analysis_df = youth_df.merge(culture_center_df, on="C1_NM", how="inner")

# 4. 산점도
plt.figure(figsize=(12, 8))
plt.scatter(analysis_df["생활문화센터 개수"], analysis_df["청년흡수지수"])

for _, r in analysis_df.iterrows():
    plt.text(r["생활문화센터 개수"], r["청년흡수지수"], r["C1_NM"], fontsize=8)

plt.axhline(0, linestyle="--")
plt.axvline(analysis_df["생활문화센터 개수"].mean(), linestyle="--")

plt.xlabel("생활문화센터 수")
plt.ylabel("청년흡수지수")
plt.title("생활문화센터 수와 청년흡수지수")

plt.show()

### 2-1-4. 국내 문화체육관광 분야 영화관 시설 데이터

In [ ]:
import pandas as pd

theater_df = pd.read_csv("./data/KC_497_DMSTC_MCST_THEART_2025.csv")

theater_df.head()

In [ ]:
theater_df[["sido_nm", "sgg_nm"]]

In [ ]:
theater_df = (
    theater_df.groupby(["sido_nm", "sgg_nm"]).size().reset_index(name="영화관 개수")
)
theater_df

In [ ]:
theater_use = theater_df.copy()

# 1-1. 행정명 정규화
theater_use["C1_NM"] = theater_use["sgg_nm"]

# (1) ○○시○○구 → ○○시
theater_use["C1_NM"] = theater_use["C1_NM"].str.replace(r"(.*?시).*", r"\1", regex=True)

# (2) 서울 구 → 서울시
theater_use.loc[theater_use["C1_NM"].str.endswith("구"), "C1_NM"] = "서울시"

# 1-2. 시군 단위 영화관 개수 합산
theater_city = theater_use.groupby("C1_NM", as_index=False)["영화관 개수"].sum()
theater_city

In [ ]:
youth_df = ranked[["C1_NM", "평균_청년순이동률"]].copy()

analysis_df = youth_df.merge(theater_city, on="C1_NM", how="inner")

In [ ]:
# 영화관 개수 × 청년 순이동률 그래프
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

plt.scatter(analysis_df["영화관 개수"], analysis_df["평균_청년순이동률"], alpha=0.7)

for _, r in analysis_df.iterrows():
    plt.text(r["영화관 개수"], r["평균_청년순이동률"], r["C1_NM"], fontsize=8)

plt.axhline(0, linestyle="--")
plt.axvline(analysis_df["영화관 개수"].mean(), linestyle="--")

plt.xlabel("영화관 개수")
plt.ylabel("평균 청년 순이동률")
plt.title("영화관 개수와 청년 순이동률")

plt.tight_layout()
plt.show()

# 유튜브 댓글 크롤링 후 워드 클라우드, LDA 생성

## https://www.youtube.com/watch?v=kxkacVPgTIk

### 유튜브 크롤링

In [ ]:
baseUrl = "https://www.googleapis.com/youtube/v3/commentThreads"

part = "snippet"
videoId = "kxkacVPgTIk"
key = os.getenv("YOUTUBE_API_KEY")
maxResults = 100
textFormat = "plainText"
nextPageToken = None

dataList = []
max_rep = 50

for i in range(max_rep):
    url = f"{baseUrl}?part={part}&videoId={videoId}&key={key}&maxResults={maxResults}&textFormat={textFormat}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        nextPageToken = data.get("nextPageToken")

        for i in range(len(data["items"])):
            dataList.append(data["items"][i]["snippet"]["topLevelComment"]["snippet"])

        if nextPageToken is None:
            break

print(len(dataList))

df = pd.DataFrame(dataList)

### 워드클라우드

In [ ]:
import re
from kiwipiepy import Kiwi
from collections import Counter

kiwi = Kiwi()
word_list = []

for sent in df["textOriginal"]:
    clean_sent = re.sub("[^0-9a-zA-Z가-힣\s]", "", sent)
    tokens = kiwi.analyze(clean_sent)[0][0]
    sub_list = []

    for token in tokens:
        word = token.form
        pos = token.tag

        if pos == "NNG" and len(word) > 1:
            sub_list.append(word)

    word_list.extend(sub_list)

print(len(word_list))

counter = Counter(word_list)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import koreanize_matplotlib

wc = WordCloud(
    font_path="C:\Windows\Fonts\malgun.ttf",
    background_color="white",
    width=800,
    height=400,
    prefer_horizontal=1.0,
)

wc.generate_from_frequencies(counter)

plt.figure(figsize=(8, 8))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

### LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    max_df=0.1, min_df=2, max_features=1000, ngram_range=(1, 2)
)

feat_vec = count_vectorizer.fit_transform(word_list)
feat_vec.shape

In [ ]:
import pandas as pd

df_vec = pd.DataFrame(
    feat_vec.toarray(), columns=count_vectorizer.get_feature_names_out()
)
df_vec.head()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=3)
lda.fit(feat_vec)

In [ ]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
pyLDAvis.display(vis)

## https://www.youtube.com/watch?v=ZghVVrcYAuc

### 유튜브 크롤링

In [ ]:
import os
import requests

baseUrl = "https://www.googleapis.com/youtube/v3/commentThreads"

part = "snippet"
videoId = "ZghVVrcYAuc"
key = os.getenv("YOUTUBE_API_KEY")
maxResults = 100
textFormat = "plainText"
nextPageToken = None

dataList2 = []
max_rep = 50

for i in range(max_rep):
    url = f"{baseUrl}?part={part}&videoId={videoId}&key={key}&maxResults={maxResults}&textFormat={textFormat}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        nextPageToken = data.get("nextPageToken")

        for i in range(len(data["items"])):
            dataList2.append(data["items"][i]["snippet"]["topLevelComment"]["snippet"])

        if nextPageToken is None:
            break

print(len(dataList2))

df = pd.DataFrame(dataList2)

### 워드클라우드

In [ ]:
import re
from kiwipiepy import Kiwi
from collections import Counter

kiwi = Kiwi()
word_list2 = []

for sent in df["textOriginal"]:
    clean_sent = re.sub("[^0-9a-zA-Z가-힣\s]", "", sent)
    tokens = kiwi.analyze(clean_sent)[0][0]
    sub_list = []

    for token in tokens:
        word = token.form
        pos = token.tag

        if pos == "NNG" and len(word) > 1:
            sub_list.append(word)

    word_list2.extend(sub_list)

print(len(word_list2))

counter2 = Counter(word_list2)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import koreanize_matplotlib

wc = WordCloud(
    font_path="C:\Windows\Fonts\malgun.ttf",
    background_color="white",
    width=800,
    height=400,
    prefer_horizontal=1.0,
)

wc.generate_from_frequencies(counter2)

plt.figure(figsize=(8, 8))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

### LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    max_df=0.1, min_df=2, max_features=1000, ngram_range=(1, 2)
)

feat_vec = count_vectorizer.fit_transform(word_list2)
feat_vec.shape

In [ ]:
import pandas as pd

df_vec = pd.DataFrame(
    feat_vec.toarray(), columns=count_vectorizer.get_feature_names_out()
)
df_vec.head()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=3)
lda.fit(feat_vec)

In [ ]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
pyLDAvis.display(vis)

## https://www.youtube.com/watch?v=PfseJBL9Vl0

### 유튜브 크롤링

In [ ]:
import os
import requests

baseUrl = "https://www.googleapis.com/youtube/v3/commentThreads"

part = "snippet"
videoId = "PfseJBL9Vl0"
key = os.getenv("YOUTUBE_API_KEY")
maxResults = 100
textFormat = "plainText"
nextPageToken = None

dataList3 = []
max_rep = 50

for i in range(max_rep):
    url = f"{baseUrl}?part={part}&videoId={videoId}&key={key}&maxResults={maxResults}&textFormat={textFormat}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        nextPageToken = data.get("nextPageToken")

        for i in range(len(data["items"])):
            dataList3.append(data["items"][i]["snippet"]["topLevelComment"]["snippet"])

        if nextPageToken is None:
            break

print(len(dataList3))

df = pd.DataFrame(dataList3)

### 워드클라우드

In [ ]:
from konlpy.tag import Okt
import re
from kiwipiepy import Kiwi
import re

kiwi = Kiwi()
word_list3 = []

for sent in df["textOriginal"]:
    clean_sent = re.sub("[^0-9a-zA-Z가-힣\s]", "", sent)
    tokens = kiwi.analyze(clean_sent)[0][0]
    sub_list = []

    for token in tokens:
        word = token.form
        pos = token.tag

        if pos == "NNG" and len(word) > 1:
            sub_list.append(word)

    word_list3.extend(sub_list)

print(len(word_list3))

counter3 = Counter(word_list3)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import koreanize_matplotlib

wc = WordCloud(
    font_path="C:\Windows\Fonts\malgun.ttf",
    background_color="white",
    width=800,
    height=400,
    prefer_horizontal=1.0,
)

wc.generate_from_frequencies(counter3)

plt.figure(figsize=(8, 8))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

### LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    max_df=0.1, min_df=2, max_features=1000, ngram_range=(1, 2)
)

feat_vec = count_vectorizer.fit_transform(word_list3)
feat_vec.shape

In [ ]:
import pandas as pd

df_vec = pd.DataFrame(
    feat_vec.toarray(), columns=count_vectorizer.get_feature_names_out()
)
df_vec.head()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=3)
lda.fit(feat_vec)

In [ ]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
pyLDAvis.display(vis)